# Exercise Sheet 5: Electrodes Electronics

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import bci_minitoolbox as bci
from scipy import signal as signal
import Exercise5_helper as helper
from music import *


## Task 1: noise & signal models (4 points)
Simulate and plot noise & a simple alpha-oscillation model on a timescale of $5s$ with a sampling frequency of $1kHz$.

a) Generate a function *noise_w* that implements gaussian white noise with the variance $\sigma_n$ as an input parameter. White noise can be simply generated using np.random functions and the gaussian one would use the np.random.normal or np.random.randn.

b) Use the white noise function to produce pink noise (1/f) *noise_p* by frequency filtering it in the spectral domain. Therefor do a fourier transformation (np.fft.rfft) of the white noise, get the corresponding frequencies (np.fft.rfftfreq) and then multiply the fourier transfromed signal by $\frac{1}{f}$ (the factor $\frac{1}{f^2}$ is defined in the power spectrum which leads to $\sqrt{\frac{1}{f^2}}=\frac{1}{f}$ in the amplitude spectrum). As the DC part ($f=0$) would lead to a division by zero, you can simply divide the coresponding fft value by 1 instead.
Then transform the signal back to time domain (np.fft.irfft).

c) Do the same as for the pink noise to generate a simulated alpha oscillation *x_alpha* by tranformation of white noise to the frequency domain, spectral filtering and then transformation back to the time domain. For the shape in the frequency domain, use a peak function similar to that found in EEG.  A gaussian peak from 8 to 13 Hz with a standard deviation of $\frac{1}{10}$ of it's window (8-13Hz) width and the function *scipy.signal.gaussian(N,std)* as an approximation to the peak of an alpha oscillation in the frequency spectrum or similar is sufficient.

Plot all three noise & signal models into one plot first in the time domain and then in a secon plot in the frequency domain^(PSD with welch algorithm).

## Task 2 EEG simulation (4 points)

a) Generate a signal 

$v(t)=\alpha_s s(t)+\alpha_w n_w(t)+\alpha_p n_p(t)$, 

where $s(t)$ is the simulation an alpha osicllation *s_alpha*, $w_n$ is white noise and $p_n$ is pink noise. The $\alpha$s are the corresponding weights to account for individual signal powers. Plot the time course for a single channel of length $5s$ with a sampling frequency of 1kHz. Do a frequency transformation using the fourier transform and plot the power spectrum. Tune your weights $\alpha$ to get a roughly EEG-like spectrum.

b) Use the leadfield of one dipole of your choice (without amplifier input impedance) and simulate the scalp potential $v$ of an alpha oscillation in that source *s* by multiplying it with the leadfield $x(t)=L_i^T s(t)$. You can also load the scalp pattern produced by dipole $i=2081$ in $p=[0, \sqrt[]\frac{2}{3}, \sqrt[]\frac{1}{3} ]^T$ saved in 'patternDip2081.npy' to avoid the calculations. Plot the time course for channels Cz and Oz into one plot. Also, plot the power spectrum for the two channels. You can find the channel index in the variable  'clab.npy'.

c) Repeat task a) but this time use the scalp potential $x(t)$ produced in b) to simulate an EEG alpha oscillation. Add the noise independently to each channel allthough this might not be fully realistic.

$v(t)=x(t)+\alpha_w n_w(t)+\alpha_p n_p(t)$

Again, plot the time course and the power spectrum for channels Cz and Oz.

d) Use the scalpmap function of the BBCI minitoolbox to plot the scalp pattern of the 10 Hz component in the fourier transform of the signal of task c) and compare it to the plain scalp pattern of that source. To extract the pattern, simply take the values at the peak (f==10Hz). This can be mutliplied with the source activity.

*Hint: You can use the exercise5_helper if you haven't succeeded with task 1!*

## Task 3: The MUSIC algorithm (3 points)
In EEG source reconstruction one of the most straight-forward approaches is to estimate the most probable source of a signal by finding the closest possible soulution to measured scalp potentials from a discrete set of dipoles. To this extent, the leadfield $L$ with dimensions NoChannels x NoDipoles x 3 is used for every dipole location to approximate the measured potential $v$ and then the closest mostly in a $L_2$ norm is expected to be the most probable source.

Luckily, the MUSIC algorithm does that for you and you have the function provided. The function returns the most probable source location index imax, it's potential vmax it produces along with the orientation/moment of the optimal dipole dip_mom and other parameters.

The true source ID is 2499.

There are EEG simulations of two different noise levels in the files "alphascalphighSNR.npy" and "alphascalplowSNR.npy". The true source without noise can be simulated by v_sim=L[:,iDip,:], where *iDip* is the true source ID 2499. You need to accord for the right orientation using the dotproduct with gridnorm[:,iDip].

** Tasks:**

a)  Determine the index of the dipole with minimum amplitude error for each noise level in the simualted potentials *v_sim* for each of the noise levels ( v_sim = no noise, v_sim_noiseL = low noise, v_sim_noiseH = high noise). Use the music algorithm and also calculate the localization error as the Euclidean distance between the source found (dip_loc) and the real position of the source (the right entry of gridpos).

b) Investigate the simulated scalp potentials *v_sim* with the *bci.scalpmap* function of the bci_minitoolbox and try to explain the different results in source localization.

c) Load the scalp pattern *alphascalp.npy* which originates in real EEG data taken from a motor imagery experiment. The PSD values where tansformed to amplitudes (square root) and normalized. Find the best matching source location using the music function, but this time search in the leafield *leadfield_relab.npy* that is relabeled to the different set of electrodes in the real data. Also, plot the scalmaps of the *alphascalp.npy* and compare it to the one of the dipole found. What's the difference? What could be the reason for this?

In [ ]:
iDip=2499

L=np.load('leadfield3dim.npy')
gridpos=np.load('gridpos.npy')
gridnorms=np.load('gridnorms.npy')
v_sim_noiseL=np.load('alphascalphighSNR.npy')
v_sim_noiseH=np.load('alphascalplowSNR.npy')


In [ ]:
L_relab=np.load('leadfield3dimrelab.npy')
alphascalp=np.load('alphascalp.npy')

## Task 4: LCMV - (4 points)

The linear constraint minimum variance algorithm uses a mixture of the head model information for a region of interest (ROI) at location $r_q$ and the signal covairance to extract filters that minimize the effect among sources while leading to a unit response for the source of interest.
The array of filters can be calculated by:

$W_i^T=[L_i^T C_x^{-1} L_i]^{-1}L_i^T C_x^{-1}$

a) Implement the LMCV algorithm for a given leadfield $L$ and the signal covariance $C_x$. The output should be the spatial filter matrix $W$ consisting of the filters for all sources of L.

b) Calculate the spatial filter matrix for the dataset given in 'imagVPaw.npz' and the leadfield matrix from 'leadfield_relab.npy'.

c) Reconstruct the source estimated in task 3 c) from 'alphascalp.npy' and plot the PSD in decibel using the welch algorithm. *If you did not succeed with task 3, use source number 2499.*